In [2]:
from PIL import Image

img = Image.open('c9.jpg')

# Coordinates of the top-left corner of the 10x10 block around the pixel (you need to find this manually)
pix_x, pix_y = 1224, 1024  # Replace x, y with actual coordinates

# Define the color red in RGB so the pixel can be easily identified 
red = (255, 0, 0)

# Modify a 5x5 area around the pixel
for i in range(pix_x, pix_x + 5):
    for j in range(pix_y, pix_y + 5):
        img.putpixel((i, j), red)

img.show()



In [7]:
import pyproj

# Define the RD New (Amersfoort) CRS
proj_rd_new = pyproj.Proj(init="epsg:28992")

# Define the WGS84 CRS (used for lat/lon)
proj_wgs84 = pyproj.Proj(init="epsg:4326")

# List of points with (Easting, Northing)
points_rd = [
    ("gcp0901", 72560.689, 451585.03),
    ("gcp0902", 72430.207, 451376.846),
    ("gcp0903", 72404.804, 451496.345),
    ("gcp0904", 72503.475, 451753.97),
    ("gcp0905", 72518.993, 451923.906),
    ("gcp0906", 72491.774, 451918.553),
    ("gcp0907", 72445.762, 451871.248),
    ("gcp0908", 72471.915, 451936.434),
    ("gcp0909", 72517.762, 451983.216),
    ("gcp0910", 72476.739, 451991.085),
    ("cam9",	72502.827, 452071.438)

]

# Convert RD New to WGS84 (Lat, Lon)
points_wgs84 = [(point[0], *pyproj.transform(proj_rd_new, proj_wgs84, point[1], point[2])) for point in points_rd]

points_wgs84


c:\Users\maikd\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\maikd\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\maikd\AppData\Local\Temp\ipykernel_3252\475230796.py:26: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotch

[('gcp0901', 4.185440605113732, 52.046443983534395),
 ('gcp0902', 4.183589193882612, 52.04455372046808),
 ('gcp0903', 4.1831900966069675, 52.04562381088798),
 ('gcp0904', 4.184565960094108, 52.047953646575024),
 ('gcp0905', 4.1847511173419285, 52.049483081906665),
 ('gcp0906', 4.184355680384403, 52.04943092589389),
 ('gcp0907', 4.183696463504096, 52.048998969112695),
 ('gcp0908', 4.184061908132926, 52.04958865626547),
 ('gcp0909', 4.184718858459477, 52.0500158865175),
 ('gcp0910', 4.184119021817647, 52.05008049429814),
 ('cam9', 4.184479871385774, 52.05080646824063)]

In [16]:
from math import radians, sin, cos, sqrt, atan2
import numpy as np
# Define the Haversine formula to calculate distance between two points on the Earth
def haversine(lat1, lon1, lat2, lon2):
    # Radius of Earth in meters
    R = 6371000  
    
    # Convert latitudes and longitudes from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    # Distance in meters
    distance = R * c
    return distance

# Coordinates of cam9 and gcp0906
lat_cam9, lon_cam9 = 52.05080646824063, 4.184479871385774
lat_gcp0906, lon_gcp0906 = 52.04943092589389, 4.184355680384403

# Calculate the distance
distance_meters = haversine(lat_cam9, lon_cam9, lat_gcp0906, lon_gcp0906)
distance_meters

correction = 2
distance = distance_meters + correction
height = 15

tilt = np.tanh(height/distance)
tilt_deg = np.degrees(tilt)
print("Tilt in Radians",tilt)
print("Tilt in Degrees",tilt_deg)






Radians 0.09635650916090893
Degrees 5.520821303533735


In [18]:
from math import degrees, atan2
import numpy as np
# Function to calculate azimuth between two points
def calculate_azimuth(lat1, lon1, lat2, lon2):
    dlon = radians(lon2 - lon1)
    lat1, lat2 = radians(lat1), radians(lat2)
    
    x = sin(dlon) * cos(lat2)
    y = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)
    
    initial_bearing = atan2(x, y)
    initial_bearing = degrees(initial_bearing)
    
    # Normalize the bearing to 0-360 degrees
    azimuth = (initial_bearing + 360) % 360
    return azimuth

# Calculate the azimuth between cam9 and gcp0906
azimuth_angle = calculate_azimuth(lat_cam9, lon_cam9, lat_gcp0906, lon_gcp0906)
azimuth_angle

# Calculate the counterclockwise angle by subtracting the azimuth from 360
ccw_angle = (360 - azimuth_angle) % 360
ccw_angle
print("Azimuth in Radians", np.radians(ccw_angle))
print("Azimuth in Degrees",ccw_angle)
print("The camera focal lenght is 12.5mm")

Azimuth in Radians 3.0861259353600228
Azimuth in Degrees 176.8219911419928
The camera focal lenght is 12.5mm


In [23]:
# Define the angles in radians (replace with actual values)
# phi = -5.53  #in degrees
# k = 176.821  #in degrees
# w = 0 

phi = np.radians(-5.53)  #in degrees
k = np.radians(176.821)  #in degrees
w = np.radians(0)


# Define the matrix M using numpy arrays
M = np.array([
    [np.cos(phi) * np.cos(k), np.cos(w) * np.sin(k) + np.sin(w) * np.sin(phi) * np.cos(k), np.sin(w) * np.sin(k) - np.cos(w) * np.sin(phi) * np.cos(k)],
    [-np.cos(k) * np.sin(k), np.cos(w) * np.cos(k) - np.sin(w) * np.sin(phi) * np.sin(k), np.sin(w) * np.cos(k) + np.cos(w) * np.sin(phi) * np.sin(k)],
    [np.sin(phi), -np.sin(w) * np.cos(phi), np.cos(w) * np.cos(phi)]
])

# Output the matrix
print(M)

[[-0.9938142   0.05545555 -0.09621863]
 [ 0.05537022 -0.99846116 -0.00534408]
 [-0.09636693 -0.          0.99534588]]
